# Preprocessing data for DeepMedic

### Loading packages needed.

In [1]:
from MEDIcaTe.nii_resampling import *
from MEDIcaTe.file_folder_ops import *
from MEDIcaTe.utilities import *
from MEDIcaTe.roi_generators import *
from MEDIcaTe.visualize_labels_ct_pet import *
from MEDIcaTe.normalize import *
from random import sample
import numpy as np


We first create a "raw-data-base" explicitly for DeepMedic where all data is resampled to same size.

Here we will use the medican voxel size of the entire dataset. 
Go through all images and get voxel sizes. Then get the median of this data. 

This script assumes that the 'path_to_data' only holds nifty files and that these are named according to the convention: 
<dataset_name>_<pt_num>_<modality_code>.nii.gz

In our data these are:
dataset_name = 'HNC01'
pt_num = from 000 up to 835
modality code = 0000 for CT and 0001 for PET

### Find median voxel size in data set

In [3]:
path_to_data = '/homes/kovacs/project_data/hnc-auto-contouring/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task500_HNC01/imagesTr'
x_all = []
y_all = []
z_all = []

# Run through all patient CT cases and get the pixel dimensions
# Note this takes a while if you have many cases. 
# For me: approx 1,5 minutes for 836 cases.
for i,f in enumerate(listdir(path_to_data)):
    if f[-11:-7] == '0000':
        x, y, z = find_pix_dim_with_orientation(join(path_to_data,f)) # 
        x_all.append(x)
        y_all.append(y)
        z_all.append(z)

In [12]:
median_x = np.median(x_all)
median_y = np.median(y_all)
median_z = np.median(z_all)

print(f'The selected resample voxel size is (x,y,z) = ({median_x}, {median_y}, {median_z})')

The selected resample voxel size is (x,y,z) = (-0.9765625, 0.9765625, 2.0)


We now have the size we wish to resample to. 
We perform the resampling in the following step.

### Resample the image data
Note, that nibabel.processing.resample does not take negative numbers as input on the voxel_size parameter, so the images have to manualle flipped to (L,A,S) orientation after resampling. 

In [ ]:
path_to_data = '/homes/kovacs/project_data/hnc-auto-contouring/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task500_HNC01/imagesTr'
output_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images'
# hard-code numbers if above already run once.
median_x = -0.9765625
median_y = 0.9765625
median_z = 2.0
voxel_size = (median_x, median_y, median_z)
for i,f in enumerate(listdir(path_to_data)): # go through all files on path
    file_to_resample = join(path_to_data,f)
    output_file = join(output_path,f)
    if isfile(output_file):
        pix_dims = find_pix_dim_with_orientation(output_file) # checking orientation of already resampled case
    if not isfile(output_file): # only process files if not created aready
        resample(join(path_to_data,f), output_path, voxel_size, verbose = True)
    elif pix_dims != voxel_size: # reprocess if the voxel-size of existing case is not correct
        resample(join(path_to_data,f), output_path, voxel_size, verbose = True)
    else:
        print(f'Apparent success on {f}.')


### Resample the labels


In [ ]:
input_path = '/homes/kovacs/project_data/hnc-auto-contouring/nnUNet/nnUNet_raw_data_base/nnUNet_raw_data/Task500_HNC01/labelsTr/'
output_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/labels'
median_x = -0.9765625
median_y = 0.9765625
median_z = 2.0
voxel_size = (median_x, median_y, median_z)

for case in listdir(input_path):
    cur_case = join(input_path,case)
    resample(cur_case, output_path, voxel_size, order = 0, verbose = True)

### Orientation check
To be on the safe side, we make an extra check, if all orientations are (L,A,S). In case not, the following script prints an alarm. 

In [10]:
# Control orientation of all the resampled images
# Outputting alarm-message every time orientation is not ('L', 'A', 'S')
path_resampled_dat = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images'
for i,f in enumerate(listdir(path_resampled_dat)): # go through all files on path
    k = 0
    cur_file = join(path_resampled_dat,f)
    orientation = control_nii_orientation(cur_file)
    if orientation != ('L','A','S'):
        print(f'Alarm: {f} had orientation {orientation}.')
        k += 1
if k > 0:
    print(f'{k} alarms were produced. Return to what went wrong.')
else:
    print(f'{k} alarms were produced. Looking good to proceed.')


0 alarms were produced. Looking good to proceed.


In [30]:
# Do the same for the labels

# Control orientation of all the resampled labels
# Outputting alarm-message every time orientation is not ('L', 'A', 'S')
path_resampled_dat = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/labels'
for i,f in enumerate(listdir(path_resampled_dat)): # go through all files on path
    k = 0
    cur_file = join(path_resampled_dat,f)
    orientation = control_nii_orientation(cur_file)
    if orientation != ('L','A','S'):
        print(f'Alarm: {f} had orientation {orientation}.')
        k += 1
if k > 0:
    print(f'{k} alarms were produced. Return to what went wrong.')
else:
    print(f'{k} alarms were produced. Looking good to proceed.')

0 alarms were produced. Looking good to proceed.


### Generate ROIs
Before we can normalize we need some ROIs. 
These are created in agreement with Kamnitas et al.
The ROI is based on smoothing the PET with $\sigma = 5$ and then thresholding with all intensities $<0.2$ SUV set to air and the rest as the body. 
The normalization and sampling for DeepMedic will be based on this ROI. 

In [45]:
# Generetng the ROIS
destination_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/rois'
pt_images_nifty = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/'
#generate_rois_dm_pt(pt_image_nifty, destination_path)

# go through all cases and generate the roi nifty's
for i, nifty_image in enumerate(listdir(pt_images_nifty)):
    cur_nifty_image = join(pt_images_nifty, nifty_image)
    if nifty_image[-11:-7] == '0001' and nifty_image == 'HNC01_521_0001.nii.gz': # only run for PET files
        #print(nifty_image)
        generate_rois_dm_pt(cur_nifty_image, destination_path, clobber = True)

In [ ]:
# Visualizing 20 randomly sampled cases 
final_path = '/homes/kovacs/project_scripts/hnc_segmentation/MEDIcaTe/deepMedic/figures/roi_check'
ct_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/'
pet_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/'
roi_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/rois/'

random_20_cases = sample(listdir(ct_path),20)

for case in random_20_cases:
    case_id = case[:-12]
    ct_fname_path = join(ct_path,f'{case_id}_0000.nii.gz')
    pet_fname_path = join(pet_path,f'{case_id}_0001.nii.gz')
    roi_fname_path = join(roi_path,f'{case_id}_0001_roi.nii.gz')
    # note that this version of visualization is not optimied ant does take
    # about 10 minutes to run for each case.
    visualization(final_path, ct_path=ct_fname_path, 
                  pet_path=pet_fname_path, label1_path=roi_fname_path, 
                  label2_path=roi_fname_path) # , slice_range = list(range(0, 10))


In [ ]:
# visualize case HNC01_437_0001.nii.gz which is failing
final_path = '/homes/kovacs/project_scripts/hnc_segmentation/MEDIcaTe/deepMedic/figures/roi_check'
ct_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/HNC01_521_0000.nii.gz'
pet_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/HNC01_521_0001.nii.gz'
roi_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/rois/HNC01_521_0001_roi.nii.gz'
visualization(final_path, ct_path=ct_fname_path, 
                  pet_path=pet_fname_path,
                  label1_path=roi_fname_path,
                  label2_path=roi_fname_path)

In [48]:
ct_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/HNC01_521_0000.nii.gz'
pet_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/HNC01_521_0001.nii.gz'
roi_fname_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/rois/HNC01_521_0001_roi.nii.gz'
input_mean_roi, input_std_roi = get_nii_mean_std_in_roi(input_nii=pet_fname_path, input_roi=roi_fname_path)
print(f'input_mean_roi = {input_mean_roi}, input_std_roi = {input_std_roi}')

input_mean_roi = 1.4963958237389199, input_std_roi = 1.9662202375742805


### Normalize images
The PET and CT images are normalized to 0 mean and unit variance in accorance with DeepMedic documentation.

Start by getting the normalization parameters:

In [52]:
# add paths to all images, rois and where 
pt_images_nifty = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/'
roi_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/rois'

# case HNC01_437_0001.nii.gz is failing ??? 

# get mean and std for CT
input_mean_roi_ct = []
input_std_roi_ct = []
for i, nifty_image in enumerate(listdir(pt_images_nifty)):
    cur_nii_image = join(pt_images_nifty, nifty_image)
    if nifty_image[-11:-7] == '0000': #only do anything if CT
        cur_roi = join(roi_path, f'{basename(cur_nii_image[:-12])}_0001_roi.nii.gz')
        input_mean_roi, input_std_roi = get_nii_mean_std_in_roi(input_nii=cur_nii_image, input_roi=cur_roi)
        input_mean_roi_ct.append(input_mean_roi)
        input_std_roi_ct.append(input_std_roi)


# get mean and std for PET
input_mean_roi_pet = []
input_std_roi_pet = []
for i, nifty_image in enumerate(listdir(pt_images_nifty)):
    cur_nii_image = join(pt_images_nifty, nifty_image)
    if nifty_image[-11:-7] == '0001': #only do anything if CT
        cur_roi = join(roi_path, f'{basename(cur_nii_image[:-12])}_0001_roi.nii.gz')
        input_mean_roi, input_std_roi = get_nii_mean_std_in_roi(input_nii=cur_nii_image, input_roi=cur_roi)
        input_mean_roi_pet.append(input_mean_roi)
        input_std_roi_pet.append(input_std_roi)

# get final metrics
ct_mean_gl = np.mean(input_mean_roi_ct)
ct_std_gl = np.mean(input_std_roi_ct)
pet_mean_gl = np.mean(input_mean_roi_pet)
pet_std_gl = np.mean(input_std_roi_pet)

print(ct_mean_gl)
print(ct_std_gl)
print(pet_mean_gl)
print(pet_std_gl)


-86.70951920247971
449.6891469125435
1.454322559282762
1.9064418631810773


And we find the statistics:

ct_mean_gl = -86.70951920247971

ct_std_gl = 449.6891469125435

pet_mean_gl = 1.454322559282762

pet_std_gl = 1.9064418631810773

Then normalize using the obtained values

In [54]:
# Normalize all files
pt_images_nifty = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train_raw_data_base/images/'
destination_path = '/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/images_normalized'

'''
in case not already set, use this bit (for my HNC_01 data)

ct_mean_gl = -86.70951920247971
ct_std_gl = 449.6891469125435
pet_mean_gl = 1.454322559282762
pet_std_gl = 1.9064418631810773
'''

for f in listdir(pt_images_nifty):
    input_nii = join(pt_images_nifty, f)
    if f[-11:-7] == '0000':
        norm_0mean_1variance(input_nii, destination_path, ct_mean_gl, ct_std_gl)
    elif f[-11:-7] == '0001':
        norm_0mean_1variance(input_nii, destination_path, pet_mean_gl, pet_std_gl)


print('I executed w.o. errors')
# TODO: QA of content in /homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifty/d_train/images_normalized

I executed w.o. errors


### Generate config files
Finally we can generate the config files necessary to run DeepMedic.

Since we are training 5-fold cross-validation, need to generate config-files for each of the five folds. Eventually we will use majority voting between predictions of the 5 models in order get the final segmentation of DeepMedic.

Note: For the data_split I am using the same structure as in nnUNet, which is documented here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/common_questions.md under "Creating and managing splits."

In [ ]:
'''
Before doing this, I reorganized my files from structure

-- data
    -- d_train
        -- images_normalized
            HNC01_000_0000.nii.gz
            HNC01_000_0001.nii.gz
            HNC01_001_0000.nii.gz
            HNC01_001_0001.nii.gz
            ...
        -- rois
            HNC01_000_0001_roi.nii.gz
            HNC01_001_0001_roi.nii.gz
            ...
        -- labels
            HNC01_000.nii.gz
            HNC01_001.nii.gz
            ...

To the structure
-- data_as_deepmedic
    -- d_train
        -- HNC01_000
            HNC01_000_0000.nii.gz
            HNC01_000_0001.nii.gz
            HNC01_000_0001_roi.nii.gz
            HNC01_001.nii.gz
        -- HNC01_001
            ...
        ...
To match how DeepMedic organizes files. 
I am not including this script, since it depends on how your data is originally organized.
'''

In [3]:
dm_train_dat_path = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifti_as_deepmedic/train_normalized'
splits_file_path = '/homes/kovacs/project_data/hnc-auto-contouring/nnUNet/nnUNet_preprocessed/Task500_HNC01/splits_final.pkl'
config_path_train = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/config_files/train'
config_path_valid = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/config_files/train/validation'

#docker_train_dat_path = pathlib.Path(dm_train_dat_path)
#docker_train_dat_path = pathlib.Path(*docker_train_dat_path.parts[4:])
#docker_train_dat_path = str(f'/{docker_train_dat_path}')

splits = load_pickle(splits_file_path)

def generate_config_files(train_or_valid, destination_path):
    for fold in np.arange(len(splits)):
        files_ct = []
        files_pt = []
        files_roi = []
        files_label = []
        if train_or_valid == 'train':
            dat = splits[fold]['train']
        elif train_or_valid == 'val':
            dat = splits[fold]['val']

        absolute_subtring = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/'
        relative_substring = '../../'
        
        for f in dat:
            fname_ct = join(dm_train_dat_path,f,f'{f}_0000.nii.gz')
            fname_pet = join(dm_train_dat_path,f,f'{f}_0001.nii.gz')
            fname_roi = join(dm_train_dat_path,f,f'{f}_0001_roi.nii.gz')
            fname_label = join(dm_train_dat_path,f,f'{f}.nii.gz')
            #ct
            if isfile(fname_ct):
                files_ct.append(fname_ct.replace(absolute_subtring,relative_substring))
            else: 
                print('ERROR: CT file did not exist')
            #pet
            if isfile(fname_pet):
                files_pt.append(fname_pet.replace(absolute_subtring,relative_substring))
            else: 
                print('ERROR: PET file did not exist')
            #roi
            if isfile(fname_roi):
                files_roi.append(fname_roi.replace(absolute_subtring,relative_substring))
            else: 
                print('ERROR: ROI file did not exist')
            #label
            if isfile(fname_label):
                files_label.append(fname_label.replace(absolute_subtring,relative_substring))
            else: 
                print('ERROR: Label file did not exist')
        #print(files_fold0_ct)
        all_im_roi_labs = [files_ct, files_pt, files_roi, files_label]
        out_files = ['ct', 'pet', 'roi', 'labels']
        for i, modality in enumerate(out_files):
            outF = open(join(destination_path,f'{modality}_fold{fold}.cfg'), 'w')
            for line in all_im_roi_labs[i]:
            # write line to output file
                outF.write(line)
                outF.write("\n")
            outF.close()

generate_config_files(train_or_valid = 'train', destination_path = config_path_train)
generate_config_files(train_or_valid = 'val', destination_path = config_path_valid)


### Casting the data to float32 and int16
Running DeepMedic it turned out a large part of the training time was spent on loading the data.
This is due to saving everything as float64. 
To improve efficiency and training time, I concvert images to float32 and label files to int16. 

In [ ]:
'''
This bit is the same as file MEDIcaTe/deepMedic/conv.py which is used to run process in background (and go home to get some sleep :) )
Had to rerun this because I did something wrong at first attempt so I converted to multiprocess to avoid the wait.

Note: ensure to create destination path 'path_to_data_dst' before running
'''
from multiprocessing import Pool
path_to_data_src = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifti_as_deepmedic/train'
path_to_data_dst = '/media/bizon/data2tb/deep_medic/homes/kovacs/project_data/hnc-auto-contouring/deepMedic/data_nifti_as_deepmedic/train_r'

def process_image(name):
    cur_src_path = join(path_to_data_src,name)
    cur_dst_path = join(path_to_data_dst,name)
    if not isdir(cur_dst_path):
        os.mkdir(cur_dst_path)
    for fi in listdir(cur_src_path):
        cur_src_file_path = join(cur_src_path,fi)
        cur_dst_file_path = join(cur_dst_path,fi)
        if not isfile(cur_dst_file_path):
            if (fi[-11:-7] == '0000') or (fi[-11:-7] == '0001'):
                convert_nii_to_float32(cur_src_file_path,cur_dst_file_path)
                print(f'Converted {fi} to float32')
            else: 
                convert_nii_to_int16(cur_src_file_path,cur_dst_file_path)
                print(f'Converted {fi} to int16')

pool = Pool()
pool.map(process_image, listdir(path_to_data_src))

